In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def obter_links_municipios(pagina):
    url = f'https://www1.folha.uol.com.br/remf/?page={pagina}'
    resposta = requests.get(url)
    sopa = BeautifulSoup(resposta.text, 'html.parser')
    links = sopa.select('a[href^="/remf/municipio/"]')
    urls_municipios = ['https://www1.folha.uol.com.br' + link['href'] for link in links]
    return urls_municipios

def obter_dados_municipio(url):
    resposta = requests.get(url)
    sopa = BeautifulSoup(resposta.text, 'html.parser')
    
    # Extração do nome do município
    nome = sopa.find('h1').text.strip()
    
    # Extração dos indicadores de eficiência
    eficiencia = {}
    indicadores = sopa.find_all('div', class_='indicador')
    for indicador in indicadores:
        titulo = indicador.find('h2').text.strip()
        valor = indicador.find('span', class_='valor').text.strip()
        eficiencia[titulo] = valor
    
    # Extração dos dados demográficos e outros indicadores
    dados = {}
    itens = sopa.find_all('div', class_='item')
    for item in itens:
        titulo = item.find('h3').text.strip()
        valor = item.find('p').text.strip()
        dados[titulo] = valor
    
    # Combina todas as informações em um único dicionário
    dados_municipio = {'Nome': nome}
    dados_municipio.update(eficiencia)
    dados_municipio.update(dados)
    
    return dados_municipio

def coletar_dados_e_salvar():
    todos_dados = []
    pagina = 1
    municipio_count = 0
    arquivo_count = 1

    while True:
        links_municipios = obter_links_municipios(pagina)
        if not links_municipios:
            break
        
        for link in links_municipios:
            dados = obter_dados_municipio(link)
            todos_dados.append(dados)
            municipio_count += 1
            
            # Salvar a cada 100 municípios
            if municipio_count % 100 == 0:
                df = pd.DataFrame(todos_dados)
                caminho_arquivo = f'C:\\municipios_batch_{arquivo_count}.xlsx'
                df.to_excel(caminho_arquivo, index=False)
                print(f'Salvo: {caminho_arquivo}')
                arquivo_count += 1
                todos_dados = []
            
            time.sleep(1)  # Respeitar o servidor com uma pausa
        
        pagina += 1

    # Salvar quaisquer dados restantes
    if todos_dados:
        df = pd.DataFrame(todos_dados)
        caminho_arquivo = f'C:\\municipios_batch_{arquivo_count}.xlsx'
        df.to_excel(caminho_arquivo, index=False)
        print(f'Salvo: {caminho_arquivo}')

if __name__ == '__main__':
    coletar_dados_e_salvar()
